In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from io import StringIO
from selenium.webdriver.common.keys import Keys
import pandas as pd
from datetime import datetime
import os
import numpy as np

In [2]:
start_time = datetime.now()

In [3]:
dir = 'Y:\delete\Cosmetic Ingridient Database'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

In [4]:
# create webdriver object
ser = Service("C:/Users/AIMPAI0/Downloads/chromedriver.exe")
op = webdriver.ChromeOptions()
prefs = {"download.default_directory" : "Y:\\delete\\Cosmetic Ingridient Database"}
op.add_experimental_option("prefs",prefs)
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()  
driver.get("https://data.europa.eu/data/datasets/cosmetic-ingredient-database-2-list-of-substances-prohibited-in-cosmetic-products?locale=en")
time.sleep(10)
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[7]/div[2]/div/button').click()
time.sleep(10)
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[6]/div/div[2]/div/div/div[2]/div/button/span').click()
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[6]/div/div[2]/div/div/div[2]/div/div/span[2]/a/small').click()
time.sleep(10)
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[8]/div/div[2]/div/div/div[2]/div/button/span').click()
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[8]/div/div[2]/div/div/div[2]/div/div/span[2]/a/small').click()
time.sleep(10)
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[11]/div/div[2]/div/div/div[2]/div/button/span').click()
driver.find_element(by=By.XPATH, value='/html/body/div/div[4]/div[2]/div/div[2]/div[4]/div/ul/div/div[11]/div/div[2]/div/div/div[2]/div/div/span[2]/a/small').click()
time.sleep(10)
driver.close()

In [87]:
Ydf1 = pd.read_csv('Y:/delete/Cosmetic Ingridient Database/COSING_Annex V_v2.csv',skiprows=8)
df1 = df1[['CAS Number']]
df1['CAS Number'] = df1['CAS Number'].str.split('/')
df1 = df1.explode('CAS Number').reset_index(drop=True)
nan_value = float("NaN")
df1 = df1.replace(" ", nan_value)
df1 = df1.dropna()
df1 = df1.reset_index(drop=True)
df1['annex'] = 'Annex V'

df2 = pd.read_csv('Y:/delete/Cosmetic Ingridient Database/COSING_Annex III_v2.csv',skiprows=8)
df2 = df2[['CAS Number']]
df2['CAS Number'] = df2['CAS Number'].str.split('/')
df2 = df2.explode('CAS Number').reset_index(drop=True)
nan_value = float("NaN")
df2 = df2.replace(" ", nan_value)
df2 = df2.dropna()
df2 = df2.reset_index(drop=True)
df2['annex'] = 'Annex III'

df3 = pd.read_csv('Y:/delete/Cosmetic Ingridient Database/COSING_Annex II_v2.csv',skiprows=6)
df3 = df3[['CAS Number']]
df3['CAS Number'] = df3['CAS Number'].str.split('/')
df3 = df3.explode('CAS Number').reset_index(drop=True)
nan_value = float("NaN")
df3 = df3.replace(" ", nan_value)
df3 = df3.dropna()
df3 = df3.reset_index(drop=True)
df3['annex'] = 'Annex II'

CAS = pd.read_excel('Y:/delete/Chemical CAS.xlsx')
CAS['value'] = 'Annex V'
CAS = CAS.applymap(lambda x: x.strip() if isinstance(x, str) else x)

frames = [df1, df2, df3]
dataset = pd.concat(frames)
dataset = dataset.applymap(lambda x: x.strip() if isinstance(x, str) else x)

result = pd.merge(dataset, CAS, how="outer", on="CAS Number")
result.annex.fillna(result.value, inplace=True)
del result['value']
result.head(2)

,CAS Number,annex
0,65-85-0,Annex V
1,532-32-1,Annex V


In [85]:
result['Jurisdiction'] = 'Europe'
result['RegulatoryAgency'] = 'European Directorate-General'
result['TypeOfRegulation'] = 'Use'
result['Criteria'] = 'Use in Cosmetics'

result['Description'] = 'Prohibited in cosmetics products'
result.loc[result['annex'] == 'Annex V', 'Description'] = 'Not prohibited in cosmetics products'
result.loc[result['annex'] == 'Annex III', 'Description'] = 'Restricted in cosmetics products'



result['HazardClassification'] = 'High'
result.loc[result['annex'] == 'Annex V', 'HazardClassification'] = 'Low'
result.loc[result['annex'] == 'Annex III', 'HazardClassification'] = 'Moderate'

result['CauseOfClassification'] = 'Prohibited by  EU Directorate-General'
result.loc[result['annex'] == 'Annex V', 'CauseOfClassification'] = 'Not prohibited by  EU Directorate-General'
result.loc[result['annex'] == 'Annex III', 'CauseOfClassification'] = 'Restricted by  EU Directorate-General'

result['SourceName'] = 'Cosmetic Ingridient Database'
result['SourceURL'] = 'https://data.europa.eu/data/datasets/cosmetic-ingredient-database-2-list-of-substances-prohibited-in-cosmetic-products?locale=en'
result.head(2)

,CAS Number,annex,Jurisdiction,RegulatoryAgency,TypeOfRegulation,Criteria,Description,HazardClassification,CauseOfClassification,SourceName,SourceURL
0,65-85-0,Annex V,Europe,European Directorate-General,Use,Use in Cosmetics,Not prohibited in cosmetics products,Low,Not prohibited by EU Directorate-General,Cosmetic Ingridient Database,https://data.europa.eu/data/datasets/cosmetic-...
1,532-32-1,Annex V,Europe,European Directorate-General,Use,Use in Cosmetics,Not prohibited in cosmetics products,Low,Not prohibited by EU Directorate-General,Cosmetic Ingridient Database,https://data.europa.eu/data/datasets/cosmetic-...


In [ ]:
results.to_csv('Y:/delete/Reugulatory Assessment.csv',index=False)

In [88]:
end_time = datetime.now()
print('The total time taken is', (end_time - start_time))

The total time taken is 0:41:55.853379
